<a href="https://colab.research.google.com/github/ohoelske/Hausarbeit_Forschungsdatenmanagement/blob/main/210407_Implementierung_TREAT_COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python Quellcode zum Projektes "TREAT COVID-19"**

Autor: Oliver Hölsken

Version 1.0

#1. Environment erstellen

##Variabeln löschen

In [2]:
%reset -f

##Import Libraries

In [3]:
import sys
import pandas as pd
from pandas.util import hash_pandas_object
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
sns.set(color_codes=True)
sns.set_theme(style="darkgrid")
from google.colab import drive
from pandas_profiling import ProfileReport

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [4]:
material_path = '/content/drive/MyDrive/Forschungsdatenmanagement/Hausarbeit/csv.files'

##Configuration

In [5]:
%matplotlib inline

##Versioncheck

In [6]:
#Python
#sys.version_info

In [7]:
#pandas
pd.__version__

'1.1.5'

In [8]:
#seaborn
sns.__version__

'0.11.1'

In [9]:
#matplotlib
mlp.__version__

'3.2.2'

In [10]:
#numpy
np.__version__

'1.19.5'

### Versionen der Umgebung und Bibliotheken (Zusammenfassung)

|Bibliothek|Version|
|-|-|
| python | 3.7.10 |
| pandas |1.1.5|
| numpy |1.19.5|
| matplotlib |3.2.2|
| seaborn |0.11.1|
| google|2.0.3|


In [11]:
# %pip freeze

# 2. Abfragen zur Datenqualität

##V0 Datei einlesen

In [12]:
#Einlesen der v0.csv Datei als dataframe_0
df_0 = pd.read_csv('/content/drive/MyDrive/Forschungsdatenmanagement/Hausarbeit/csv_files/Kopie_Complete_Dataset_Covid_10k_Valid_V0.csv', sep=";") # ggf. ","

FileNotFoundError: ignored

In [ ]:
#Generelle Info über Einträge und Datentypen
df_0.info()

In [ ]:
#Erste Zeilen anzeigen
df_0.head()

In [ ]:
#Beispiel Index Patient Zeile 2
df_0.loc[2]

##V1 Datei einlesen

In [ ]:
#Einlesen der v1.csv Datei als dataframe
df = pd.read_csv('/content/drive/MyDrive/Forschungsdatenmanagement/Hausarbeit/csv_files/Kopie_Complete_Dataset_Covid_10k_Valid_V1.csv', sep=";") # ggf. ","

In [ ]:
#Kontrolle des Datensatzes V1
df.info()

In [ ]:
#Alle Spalten für Patient mit dem Index (1) darsellen
df.loc[1]

##Konstrukt 2A: Sind ausreichend Datenpunkte vorhanden?

###Merkmal: SARS-CoV-2 Virusnachweis (Anhand OBS_CODE und OBS_VALUE)
*Wie häufig taucht der Code 94531-1 in der Spalte OBS_Code auf ind Kombination mit Virusnachweis?*

In [ ]:
#Selektion aller Patienten mit detektierbarem Virus
df.loc[(df['OBS_CODE'] == '94531-1') & (df['OBS_VALUE'] == 'Detected (qualifier value)')]

##Merkmal: Ethnie (RACE)

In [ ]:
#Zählen der verscheinden RACE Merkmale
df['RACE'].value_counts()

In [ ]:
sns.countplot(x='RACE', data=df)
plt.title('RACE distribution')
plt.show()

##Konstrukt 2B: Besteht ein Zusammenhang zwischen Datenpunkten?

###Beispiel: Todesdatum muss NACH dem Beginn der Behandlung liegen 

In [ ]:
#print(df.dtypes)

###Überführung der Objekte in den DATE Spalten in Python datetime Objekte64

In [ ]:
#Überführung der Objekte in den DATE Spalten in Python datetime Objekte64
df["BIRTHDATE"] = pd.to_datetime(df["BIRTHDATE"])
df["DEATHDATE"] = pd.to_datetime(df["DEATHDATE"])
df["MED_START"] = pd.to_datetime(df["MED_START"])

In [ ]:
#df["OBS_DATE"] = pd.to_datetime(df["OBS_DATE"])
#df["CON_START"] = pd.to_datetime(df["CON_START"])
#df["CON_STOP"] = pd.to_datetime(df["CON_STOP"])
#df["MED_STOP"] = pd.to_datetime(df["MED_STOP"])

In [ ]:
print(df.dtypes)

###Identifikation der verstorbenen Patienten

In [ ]:
deceased_patients = df[df.DEATHDATE.notna()]

In [ ]:
deceased_patients

70 Patienten sind verstorben.

### Differenz zwischen DEATHDATE und MED_START

In [ ]:
#pd.options.mode.chained_assignment = None  # default='warn'

Die Differenz zwischen den beiden Daten wurde berechnent. Positive Werte geben an, dass die Medikation NACH dem Sterbedatum angesetzt wurde:

In [ ]:
deceased_patients['Val_Diff'] = deceased_patients['MED_START'] - deceased_patients['DEATHDATE'] 

In [ ]:
#Transformation der Val_Diff werte in Tage
deceased_patients['time_diff_days'] = deceased_patients["Val_Diff"].dt.days

In [ ]:
deceased_patients_true = deceased_patients.loc[deceased_patients['time_diff_days'] < 0]

In [ ]:
deceased_patients_true.head

Verstorbene Patienten, die Medikation nach Todesdatum erhalten haben:

In [ ]:
deceased_patients_false = deceased_patients.loc[deceased_patients['time_diff_days'] > 0]

In [ ]:
deceased_patients_false

In [ ]:
#Index der betroffen Zeilen

In [ ]:
deceased_patients_false.index.values

Diese Datenpunkte sind sehr wahrscheinlich fehlerhaft und werden weiter unten bereinigt.

###Schwangere mit dem Geschlecht M(ale)

Identifikaion aller Zeilen mit dem Eintrag 'Normal pregnancy' in der Spalte CON_DESCRIPTION

In [ ]:
pregnant_patients = df.loc[(df['CON_DESCRIPTION'] == 'Normal pregnancy')]
pregnant_patients.count

50 Zeileneinträge weisen diesen Merkmal auf.

Aufteilung in Geschlecht 

In [ ]:
pregnant_males = df.loc[(df['CON_DESCRIPTION'] == 'Normal pregnancy') & (df['GENDER'] == 'M')]
pregnant_females = df.loc[(df['CON_DESCRIPTION'] == 'Normal pregnancy') & (df['GENDER'] == 'F')]

Darstellung

In [ ]:
sns.set(rc={'figure.figsize':(7,5)})

In [ ]:
sns.countplot(x='GENDER', data=pregnant_patients)
plt.title('Distribution Pregnant patients')
plt.show()

#3. Verbesserungsvorschläge

##Die Einträge in der Spalte RACE mit den Merkmalen *Alien* oder *w* wurden entfernt.

In [ ]:
#Identifkation der Aliens
df_Aliens = df.loc[(df['RACE'] == 'Alien')]

In [ ]:
#Indizierung
df_Aliens.index.values

In [ ]:
#Bereinigung
df_cleaned = df.drop(labels=[38, 159, 456], axis=0)

In [ ]:
#Identifkation der Einträge w
df_w = df_cleaned.loc[(df['RACE'] == 'w')]

In [ ]:
#Indizierung
df_w.index.values

###Interesanterweise werden nicht alle Indices mit dem Merkmal w agezeigt. Die Indices 42-44 wurden manuell hinzugefügt:

In [ ]:
#Bereinigung
df_cleaned_2 = df_cleaned.drop(labels=[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53], axis=0)

##Entfernung der schwangeren männlichen Patienten

In [ ]:
#Identifikation
df_pregnant_m = df_cleaned_2.loc[(df_cleaned_2['CON_DESCRIPTION'] == 'Normal pregnancy') & (df_cleaned_2['GENDER'] == 'M')]

In [ ]:
#Indexierung
df_pregnant_m.index.values

In [ ]:
#Entfernung der schwangeren männlichen Patienten
df_no_pregnat_m = df_cleaned_2.drop(labels=[35,  41,  57, 140, 144, 145, 194, 202, 236, 277, 278, 279, 284,
       287, 312, 406, 412, 413, 440], axis=0)

##Entfernung der nicht aktuell verstorbenen Patienten sowie der Patienten, mit Medikation nach Todesdatum und fehlerhaften Einträgen in der Spalte OBS_Date

In [ ]:
#Identifikation laut Index aus Abschnitt Datenaufbereitung, 2B
df_incorrect_deathdate = df_no_pregnat_m.drop(labels=[11,  62,  80,  83,  89, 143, 172, 183, 201, 204, 205, 217, 235,
       246, 253, 297, 306, 315, 318, 330, 341, 347, 355, 356, 368,
       395, 399, 402, 408, 411, 414, 416, 420, 438, 439, 463, 493], axis=0)

In [ ]:
#Welche Patienten sind vor 2019 gestorben?
df_old_deathdate = df_incorrect_deathdate.loc[df_incorrect_deathdate['DEATHDATE'] < '2019-01-01']

In [ ]:
df_old_deathdate.count 

19 Patienten wuden ausgeschlossen, da sie vor 2019 verstorben waren

In [ ]:
df_old_deathdate.index.values

In [ ]:
df_clean_final = df_incorrect_deathdate.drop(labels=[107, 111, 169, 178, 180, 256, 259, 264, 270, 273, 291, 308, 319,
       387, 392, 428, 447, 462], axis=0)

In [ ]:
#Löschen von Spalten mit fehlerhaften Eintrag in OBS_date

In [ ]:
df_clean_date = df_clean_final.loc[df_clean_final['OBS_DATE'] == "4386200%"]

In [ ]:
df_clean_date2 = df_clean_final.loc[df_clean_final['OBS_DATE'] == "4386300%"]

In [ ]:
df_clean_date2

In [ ]:
#Löschen der 13. und 14. Zeile

In [ ]:
df_clean_final_v2 = df_clean_final.drop(labels=[13, 14], axis=0)

Finaler Dataframe für Version 2

In [ ]:
df_clean_final_v2 

In [ ]:
df_clean_final_v2.info()

In [ ]:
df_clean_final_v2.loc[1]

In [ ]:
df_clean_final_v2.describe

##Erstellung Version 2

In [ ]:
#df_clean_final_v2.to_csv(r'/content/drive/MyDrive/Forschungsdatenmanagement/Hausarbeit/csv_files/Kopie_Complete_Dataset_Covid_10k_Valid_V2.csv', index = False)

#4. Metadaten

In [ ]:
#Überführung der anderen Daten in das ISO 8601 Format mittels pd.to_datetime

In [ ]:
df = df_clean_final_v2

In [ ]:
df["OBS_DATE"] = pd.to_datetime(df["OBS_DATE"])
df["MED_STOP"] = pd.to_datetime(df["MED_STOP"])

In [ ]:
df.info()

In [ ]:
df.loc[1]

##Erstellung Version 3

In [ ]:
#df.to_csv(r'/content/drive/MyDrive/Forschungsdatenmanagement/Hausarbeit/csv_files/Kopie_Complete_Dataset_Covid_10k_Valid_V3.csv', index = False)

#Data Cleaning

## Fehlende Werte/Nullwerte Identifizieren



### Heatmap 

Mithilfe des Tools Seaborn können leere Zeilen im Dataframe erkannt werden.

In [ ]:
cols = df.columns
colours = ['#000099', '#ffff00'] # yellow is missing. blue is not missing.
#Erstellung einer Heatmap
sns.set(rc={'figure.figsize':(10,10)})
sns.heatmap(df[cols].isnull(), cmap=sns.color_palette(colours))

**Interpretation:**
1.   komplett leere Zeilen im Bereich um 95, 220 und 241 identifiziert.
2.   Einige Variabeln wie DEATHDATE, OBS_UNIT und CON_STOP haben relativ viele leere Zeilen.




### Fehlende Werte in %

Mittels eines for loops können die fehlenden Werte quantifiziert werden:

In [ ]:
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

###Lösungsvorschlag: Leere Zeilen löschen

In [ ]:
#Kompletter DF zeigen
#pd.set_option("display.max_rows", None, "display.max_columns", None)
#df

In [ ]:
#Zeilen identifizieren, in denen das Geburstdatum = 0 ist.
df['BIRTHDATE'].isnull().sum()

In [ ]:
#Zeilen identifizieren, in denen das Geburstdatum = 0 ist.
index = df.loc[pd.isna(df["BIRTHDATE"]), :].index
index

In [ ]:
#Indizierte Zeilen aus dem Dataframe löschen
df = df.drop(index)

In [ ]:
df.shape

3 Zeilen wurden aus dem Dataframe gelöscht.


## Duplikate

Duplizierte Reihen indentifizieren anhand von 5 Spalten

In [ ]:
#Definition der Spalten
column_names = ['BIRTHDATE', 'DEATHDATE', 'MARITAL', 'GENDER', 'BIRTHPLACE']

In [ ]:
#Identifikation der duplizierten Zeilen
duplicates = df.duplicated(subset = column_names, keep=False)
df[duplicates]

In [ ]:
#Löschen der zwei doppelten Zeilen
df = df.drop(labels=[23, 124], axis=0)

In [ ]:
#Anzeigen der neuen Zeilen-Zahl
df.shape

2 Zeilen wurden aus dem Dataframe gelöscht.

## Datenformat

Zuerst sollte sich ein Überblick über die verschiedenen Datentypen verschafft werden. Dies kann mit Hilfe der .info Funktion durchgeführt werden.

In [ ]:
df.info()

###Kategotische Werte

Die Spalten mit kategorischen Werten können über eine Funktion identifiziert werden. Dafür lässt man sich die Anzahl der Merkmalsausprägungen anzeigen.

In [ ]:
unique_counts = pd.DataFrame.from_records([(col, df[col].nunique()) for col in df.columns],
                          columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])
unique_counts

Die Spalten **MARITAL**, **RACE** und **GENDER** enthalten kategorische (<10 Merkmale) Werte und können transformiert werden.

In [ ]:
cols_to_exclude = ['DEATHDATE	', 'OBS_DATE	', 'BIRTHDATE', 'CON_STOP', 'CON_START']
for col in df.columns:
    if df[col].nunique() < 10 and col not in cols_to_exclude:
        df[col] = df[col].astype('category')

Der neue Dataframe ist kleiner und enthält nun die kategorischen Werte und ist kleiner.

In [ ]:
df.info()

###Schreibweise BIRTHPLACE

Das Format des Geburtsortes beachtet Groß- und Kleinschreibung. Dies ist potentiell problematisch für die  Maschinenlesbarkeit, v.a. von Python-basierten Applikatonen.
Außerdem fallen einige irreguläre Werte mit Sonderzeichen auf.

Das Ziel ist es, dass irreguläre Werte in der Spalte BIRTHPLACE  entfernt und eine einheitliche Schreibweise eingeführt werden(Klein-Schreibung und Entfernung von Leerzeichen)

In [ ]:
df['BIRTHPLACE']

In [ ]:
df['BIRTHPLACE_new'] = df['BIRTHPLACE'].str.lower()
df['BIRTHPLACE_new'] = df['BIRTHPLACE_new'].str.strip() # remove leading and trailing whitespace.
df['BIRTHPLACE_new'] = df['BIRTHPLACE_new'].str.replace('\\.', '') # remove period.
df

Zeilen mit irreguläre Schreibweise entfernen

In [ ]:
df_new = df.loc[df['BIRTHPLACE_new'] == 'hanoi  h√† ƒê√¥ng  vn']

In [ ]:
df_new

**h√† ƒê√¥ng  vn** wird zu **hanoi hanoi  vn**

In [ ]:
df.loc[129, 'BIRTHPLACE'] = 'Hanoi  Hanoi  VN'
df.loc[288, 'BIRTHPLACE'] = 'Hanoi  Hanoi  VN'
df.loc[129, 'BIRTHPLACE_new'] = 'hanoi  hanoi  vn'
df.loc[288, 'BIRTHPLACE_new'] = 'hanoi  hanoi  vn'

In [ ]:
df.loc[288,]

###Verbleibende Daten umformatieren

In [ ]:
df["CON_START"] = pd.to_datetime(df["CON_START"])
df["CON_STOP"] = pd.to_datetime(df["CON_STOP"])

In [ ]:
df.info()

## Irreguläre Daten / Extremwerte

###MED_DISPENSES Spalte

In [ ]:
#Histogramm
df['MED_DISPENSES'].hist(bins=100)

In [ ]:
#Deskriptive Statistik mit DataFrame.describe()
df['MED_DISPENSES'].describe()

Der höchste Wert liegt bei **583**, obwohl der Mittelwert bei 18 liegt. Sehr wahrscheinlich ein Ausreißer.

###CODE Spalten 


####CON_CODE

Die Werte für die Reihen 320, 396, 451 und 458 wurden durch die korrekten SNOMED-CT Codes ersetzt.

In [ ]:
#Alle Einträge in der Spalte CON_CODE
df['CON_CODE'].sort_values(ascending=True)

In [ ]:
df.loc[320, 'CON_CODE'] = '437931000124100' #SNOMED füe Migräne
df.loc[396, 'CON_CODE'] = '437931000124100'
df.loc[458, 'CON_CODE'] = '1551000119108' #SNOMED für Diabetische Retinopathie
df.loc[451, 'CON_CODE'] = '1511000119107' #SNOMED für periphere Neuropathie

In der Spalte wurden Objects zu Int64 (also Zahlen umgewandelt)

In [ ]:
df['CON_CODE'] = pd.to_numeric(df['CON_CODE']) 

In [ ]:
df['CON_CODE'].describe

In [ ]:
#Kompletter DF zeigen
#pd.set_option("display.max_rows", None, "display.max_columns", None)
#df

####MED_CODE: Obj --> Int64 Transformation zur besseren Lesbarkeit

In [ ]:
df['MED_CODE'] = pd.to_numeric(df['MED_CODE']) 
df['MED_CODE'] = df['MED_CODE'].astype(int)
df['MED_CODE'].sort_values(ascending=True)

##Finalen DataFrame überprüfen

In [ ]:
df.info()

In [ ]:
df.describe

In [ ]:
df.loc[1]

##Erstellung Version 4

In [ ]:
#df.to_csv(r'/content/drive/MyDrive/Forschungsdatenmanagement/Hausarbeit/csv_files/Kopie_Complete_Dataset_Covid_10k_Valid_V4.csv', index = False)

df_0 = pd.read_csv('/content/drive/MyDrive/Forschungsdatenmanagement/Hausarbeit/csv_files/Kopie_Complete_Dataset_Covid_10k_Valid_V0.csv', sep=";") # ggf. ","

#Checksum des Datensatzes überprüfen

##Datensatz laden

In [13]:
df_new = pd.read_csv('/content/drive/MyDrive/Forschungsdatenmanagement/Hausarbeit/csv_files/Kopie_Complete_Dataset_Covid_10k_Valid_V4.csv', sep=";") # ggf. ","

In [15]:
from pandas.util import hash_pandas_object
df_newhashes = hash_pandas_object(df_new)

In [16]:
df_newhashes

0       1361311020871574867
1       6562544486741711634
2      13989948102070031639
3       1723490946744185018
4       2677837615848096838
               ...         
397     6071120933425309432
398    17729717618054060606
399    11938426041472357116
400    17122329460960300879
401     8097091569756539399
Length: 402, dtype: uint64

##Wert sollte -2765201148000130798 sein

In [17]:
df_newhashes.sum()

-2765201148000130798

#Quellen
- Seaborn: https://seaborn.pydata.org/tutorial/data_structure.html
- pandas: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html

Data cleaning

1.   https://towardsdatascience.com/data-cleaning-in-python-the-ultimate-guide-2020-c63b88bf0a0d
2.   https://learn.datacamp.com/courses/cleaning-data-in-python
3. https://www.w3schools.com/python/pandas/pandas_cleaning_wrong_format.asp

